Load the data

In [59]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from sklearn.preprocessing import MinMaxScaler


In [ ]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from sklearn.preprocessing import MinMaxScaler


dataframes_list = []
directory_path = r'/kaggle/input/smemi309-final-evaluation-challenge-2023/train10/train10'

for subdir in os.listdir(directory_path):
    subdir_path = os.path.join(directory_path, subdir)

    if os.path.isdir(subdir_path):  
        word_dataframes = []

        for file in os.listdir(subdir_path):
            file_path = os.path.join(subdir_path, file)
            df = pd.read_csv(file_path)
            word_dataframes.append(df)

        dataframes_list.append(word_dataframes) # List of list of dataframes, dataframes_list[1] is the list of all data from the csv files in the addition directory


Some useful function for the pre processing

In [60]:
def nb_changes(polarity_list):
    changes = np.diff(polarity_list)
    return np.sum(changes != 0)

def nb_positive_changes(polarity_list):
    if not isinstance(polarity_list, list):
        return 0  # Return 0 if the input is not a list
    
    positive_changes = 0
    
    for i in range(1, len(polarity_list)):
        if polarity_list[i] == 1 and polarity_list[i-1] == 0:
            positive_changes += 1
    
    return positive_changes

import cv2
import numpy as np
def downsize(factor, tensor) : 

    resized_frames = np.empty((tensor.shape[0], 30, int(tensor.shape[2] * factor), int(tensor.shape[3] * factor),tensor.shape[4]))

    for i in range(tensor.shape[0]):
        for j in range(tensor.shape[1]):
            for k in range(tensor.shape[4]):

                resized_frame = cv2.resize(tensor[i, j, :, :,k], (0, 0), fx=factor, fy=factor)

                resized_frames[i, j, :, :,k] = resized_frame
    return resized_frames

pre process the data by slicing into frames and extracting features : 
number of changes, sum of polarities, positive changes and length of the list of polarities captured in a timeframe for a given pixel.
Out of these 4 features only 3 were really necessary, number of changes, sum of polarities and the length. Number of positive changes won't be used to train the model and can be removed

In [ ]:
x_range = range(0, 224)
y_range = range(0, 90)
DF2 = pd.DataFrame([(x, y) for x in x_range for y in y_range], columns=['x', 'y'])
all_results = []
num_frames=30
for list_of_df in dataframes_list:
    list_results = []  # List of results for each time interval

    # Loop over each dataframe in the inner list
    for df in list_of_df:
        results = []
        med=df.loc[:,'time'].median()
        if med <600000 :
            interval_to_slice=df[(df['time'] < 1200000)]
        elif med >2400000 :
            interval_to_slice=df[(df['time'])>1800000]
        else :
            interval_to_slice=df[(df['time']>med-600000) & (df['time']<med+600000)]
        
        
        time_intervals = np.linspace(interval_to_slice['time'].to_numpy()[0], interval_to_slice['time'].to_numpy()[len(interval_to_slice)-1], num=num_frames + 1)
        for i in range(len(time_intervals) - 1):    
            start_time, end_time = time_intervals[i], time_intervals[i + 1]
            interval_data = interval_to_slice[(interval_to_slice['time'] >= start_time) & (interval_to_slice['time'] < end_time)]
            last_polarity_per_location =interval_data.groupby(['x', 'y'])['polarity'].sum().reset_index()
            last_pol_merged=pd.merge(DF2, last_polarity_per_location, on=['x', 'y'], how='left').fillna(0)
            scaler = MinMaxScaler()
            last_pol_merged['polarity']=scaler.fit_transform(last_pol_merged['polarity'].to_numpy().reshape(-1, 1))

            count_polarity_per_location =interval_data.groupby(['x', 'y'])['polarity'].count().reset_index()
            count_pol_merged=pd.merge(DF2, count_polarity_per_location, on=['x', 'y'], how='left').fillna(0)
        

            
            grouped_data = interval_data.groupby(['x', 'y']).agg({'polarity': nb_changes}).reset_index()
            DF_merged = pd.merge(DF2, grouped_data, on=['x', 'y'], how='left')
            DF_merged['polarity'].fillna(0, inplace=True)
            DF_merged['polarity'] = scaler.fit_transform(DF_merged[['polarity']])
            
            grouped_data2=pd.DataFrame(interval_data.groupby(['x', 'y']).agg({'polarity': list}).reset_index())
            DF_merged2 = pd.merge(DF2, grouped_data2, on=['x', 'y'], how='left')
            DF_merged2['polarity'].fillna(0, inplace=True)
            DF_merged2['polarity'] = scaler.fit_transform(DF_merged2.apply(lambda row: nb_positive_changes(row['polarity']), axis=1).to_numpy().reshape(-1, 1))
            

            matrix1=last_pol_merged.pivot(index='y', columns='x', values='polarity').values 
            matrix2=count_pol_merged.pivot(index='y', columns='x', values='polarity').values 
            matrix3=DF_merged2.pivot(index='y', columns='x', values='polarity').values ##nb of changes
            matrix4=DF_merged.pivot(index='y', columns='x', values='polarity').values ##nb of changes
            

            matrix3D = np.dstack((matrix1,matrix2,matrix3,matrix4))
            results.append(matrix3D)
        list_results.append(results)
        print('ok')
    all_results.append(list_results)

In [ ]:

flattened_matrices = [matrix for list_of_results in all_results for results in list_of_results for matrix in results]

X = np.array(flattened_matrices)
X1 = X.reshape(320,30,90,224,4)
np.save('newX6.npy',X) ##30 frames, sum and nb of changes

Load the previously preprocessed data to avoid having to run the code again

In [61]:
import numpy as np
X1=np.load('/kaggle/input/tensorofdata/newX6.npy').reshape(320,30,90,224,4)

Crop to only keep the lips and global min/max scaling on the 'count' value 

In [63]:
X3 = X1[:,:, 15:65,50:170:]
X3[:,:,:,:,1]=X3[:,:,:,:,1]/np.max(X3[:,:,:,:,1])

Downsize

In [64]:
resized_frames= downsize(0.6,X3)

Plot to see the word pronunciation

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation

def update(frame):
    ax.clear()
    mean_polarity_per_location =    X3[:,:,:,:,0][0][frame]

    ax.imshow(mean_polarity_per_location, cmap='Greys_r', vmin=0, vmax=1, aspect='auto', origin='lower')
    ax.invert_yaxis()
    ax.set_title(f'Polarity Heatmap - Interval {frame + 1}')
    ax.set_xlabel('X')
    ax.set_ylabel('Y')

fig, ax = plt.subplots()
ani = FuncAnimation(fig, update, frames=30,interval=250, repeat=False)
plt.close()


from IPython.display import HTML
HTML(ani.to_jshtml())

Create a test set to see if everything works well

In [ ]:
##TEST SET
selected_ranges1 = [slice(0, 29)]
for i in range(32, 318, 32):
    selected_ranges1.append(slice(i, i + 29))

selected_matrix = np.concatenate([resized_frames[range_] for range_ in selected_ranges1], axis=0)

selected_ranges = []
for i in range(29, 318, 32):
    selected_ranges.append(slice(i, i + 3))
    
TEST=np.concatenate([resized_frames[range_] for range_ in selected_ranges], axis=0)
TESTlabels=np.array([i for i in range(10) for _ in range(3)])

GRU only

In [ ]:

from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import LSTM, Dense, TimeDistributed, Flatten,GRU, Conv2D, MaxPooling2D, Dropout, BatchNormalization
from keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.optimizers import Adam,SGD
from keras.layers import Dropout
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
from keras.regularizers import l2
from keras.layers import BatchNormalization
import tensorflow as tf
from keras.losses import SparseCategoricalCrossentropy


early_stopping = EarlyStopping(monitor='val_accuracy', patience=15, restore_best_weights=True)


custom_optimizer = Adam(learning_rate=0.0001)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(selected_matrix[:,:,:,:,[0,1,3]].reshape(290,30,30*72*3), np.array([i for i in range(10) for _ in range(29)]), test_size=0.2, random_state=35,stratify=np.array([i for i in range(10) for _ in range(29)]))


# Create the model
model = tf.keras.models.Sequential()
model.add(BatchNormalization())

model.add(GRU(units=128, return_sequences=True,dropout=0.1,recurrent_dropout=0.2, input_shape=(30, 30*72*3)))
model.add(Dropout(0.3))
model.add(GRU(units=256, return_sequences=True,dropout=0.1,recurrent_dropout=0.3,activation='relu'))
#model.add(GRU(units=128, return_sequences=True, kernel_regularizer=l2(0.015),dropout=0.2,recurrent_dropout=0.25,activation='relu'))
#model.add(GRU(units=64, return_sequences=True, kernel_regularizer=l2(0.015),recurrent_dropout=0.3,activation='relu'))

model.add(Flatten())
model.add(tf.keras.layers.Dense(128, activation='relu'))
model.add(Dropout(0.3))
model.add(tf.keras.layers.Dense(256, activation='relu'))
model.add(Dropout(0.3))


model.add(Dense(units=10))

model.compile(optimizer=custom_optimizer, loss=SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])

model.fit(X_train, y_train, epochs=150, batch_size=8, validation_data=(X_test, y_test), shuffle=True, callbacks=[early_stopping])

In [ ]:
Test=TEST[:,:,:,:,[0,1,3]]
pred=[]
for i  in range (30) : 
    result=model.predict(Test[i].reshape(1,30,30*72*3),verbose=None).argmax()
    pred.append(result)
(np.array(pred)==TESTlabels).sum()/30

CNN GRU 

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Conv3D, MaxPooling3D, Flatten, Dense, BatchNormalization, Dropout,MaxPooling2D, Conv3D, GRU,Activation
from tensorflow.keras.optimizers import Adam,SGD
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from keras.regularizers import l2
from keras.callbacks import EarlyStopping
from keras.regularizers import l2
from keras.losses import SparseCategoricalCrossentropy
early_stopping = EarlyStopping(monitor='val_accuracy', patience=15, restore_best_weights=True)



y=np.array([i for i in range(10) for _ in range(29)])
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(selected_matrix[:,:,:,:,[0,1,3]], y, test_size=0.20, random_state=41,stratify=y)

# Build the model
custom_optimizer = Adam(learning_rate=0.0001)

model = tf.keras.models.Sequential()
model.add(BatchNormalization())
model.add(Conv3D(64, (3, 2, 2), padding='same', input_shape=(30, 30, 72,3)))
model.add(Activation('relu'))
model.add(MaxPooling3D((2,2, 2)))
model.add(Dropout(0.1))

model.add(Conv3D(128, (3, 2, 2), padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling3D((2, 2, 2)))
model.add(Dropout(0.1))

model.add(Conv3D(256, (3, 2, 2), padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling3D((2, 2, 2)))
model.add(Dropout(0.1))

model.add(Reshape((3,-1)))
model.add(GRU(256, return_sequences=True, recurrent_dropout=0.1, dropout=0))
model.add(Dropout(0.4))
model.add(Flatten())

model.add(Dense(128, activation='relu'))
model.add(Activation('relu'))
model.add(Dropout(0.4))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.4))


model.add(tf.keras.layers.Dense(10))

# Compile the model
model.compile(optimizer=custom_optimizer, loss=SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=400, batch_size=8, validation_data=(X_test, y_test), callbacks=[early_stopping])

In [ ]:
Test=TEST[:,:,:,:,[0,1,3]]
pred=[]
raw_res=[]
for i  in range (30) : 
    result=model.predict(Test[i].reshape(1,30,30,72,3),verbose=None).argmax()
    pred.append(result)
(np.array(pred)==TESTlabels).sum()/30

Model seems to work on our own test set, however when passing it to the Kaggle test the result is horrendous. problem with the Kaggle test set?

Now, train the model using all available data and with the architecture that yielded the best results

In [65]:
import tensorflow as tf
from tensorflow.keras.layers import Conv3D, MaxPooling3D, Reshape, Flatten, Dense, BatchNormalization, Dropout,MaxPooling2D, Conv3D, GRU,Activation
from tensorflow.keras.optimizers import Adam,SGD
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from keras.regularizers import l2
from keras.callbacks import EarlyStopping
from keras.regularizers import l2
from keras.losses import SparseCategoricalCrossentropy
early_stopping = EarlyStopping(monitor='val_accuracy', patience=20, restore_best_weights=True)

y=np.array([i for i in range(10) for _ in range(32)])
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(resized_frames[:,:,:,:,[0,1,3]], y, test_size=0.20, random_state=41,stratify=y)

# Build the model
custom_optimizer = Adam(learning_rate=0.0001)

model = tf.keras.models.Sequential()
model.add(BatchNormalization())
model.add(Conv3D(64, (3, 2, 2), padding='same', input_shape=(30, 30, 72,3)))
model.add(Activation('relu'))
model.add(MaxPooling3D((2,2, 2)))
model.add(Dropout(0.1))

model.add(Conv3D(128, (3, 2, 2), padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling3D((2, 2, 2)))
model.add(Dropout(0.1))

model.add(Conv3D(256, (3, 2, 2), padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling3D((2, 2, 2)))
model.add(Dropout(0.1))

model.add(Reshape((3,-1)))
model.add(GRU(256, return_sequences=True, recurrent_dropout=0.3, dropout=0))
model.add(Dropout(0.5))
model.add(Flatten())

model.add(Dense(128, activation='relu'))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))


model.add(tf.keras.layers.Dense(10))

# Compile the model
model.compile(optimizer=custom_optimizer, loss=SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=400, batch_size=8, validation_data=(X_test, y_test), callbacks=[early_stopping])

Epoch 1/400
32/32 [==============================] - 7s 80ms/step - loss: 2.5067 - accuracy: 0.0781 - val_loss: 2.3030 - val_accuracy: 0.1094
Epoch 2/400
32/32 [==============================] - 2s 57ms/step - loss: 2.3576 - accuracy: 0.1133 - val_loss: 2.3016 - val_accuracy: 0.1094
Epoch 3/400
32/32 [==============================] - 2s 57ms/step - loss: 2.3227 - accuracy: 0.1289 - val_loss: 2.3008 - val_accuracy: 0.1094
Epoch 4/400
32/32 [==============================] - 2s 57ms/step - loss: 2.3709 - accuracy: 0.0820 - val_loss: 2.3002 - val_accuracy: 0.0938
Epoch 5/400
32/32 [==============================] - 2s 58ms/step - loss: 2.3415 - accuracy: 0.0820 - val_loss: 2.3004 - val_accuracy: 0.1406
Epoch 6/400
32/32 [==============================] - 2s 59ms/step - loss: 2.3098 - accuracy: 0.1133 - val_loss: 2.3011 - val_accuracy: 0.1875
Epoch 7/400
32/32 [==============================] - 2s 58ms/step - loss: 2.2969 - accuracy: 0.1367 - val_loss: 2.3006 - val_accuracy: 0.0938
Epoch 

In [66]:
model.save('89acc.keras')

In [ ]:
import os
dataframes_list = []
directory_path = r'/kaggle/input/smemi309-final-evaluation-challenge-2023/test10/test10'

for file in os.listdir(directory_path):
    file_path = os.path.join(directory_path, file)
    df = pd.read_csv(file_path)
    dataframes_list.append(df) 
x_range = range(0, 224)
y_range = range(0, 90)
DF2 = pd.DataFrame([(x, y) for x in x_range for y in y_range], columns=['x', 'y'])

In [ ]:
model = tf.keras.models.load_model('/kaggle/working/89acc.keras')

In [ ]:
import warnings

with warnings.catch_warnings():
    warnings.simplefilter("ignore")


x_range = range(0, 224)
y_range = range(0, 90)
DF2 = pd.DataFrame([(x, y) for x in x_range for y in y_range], columns=['x', 'y'])
all_results = []
num_frames=30

for df in dataframes_list:
    results = []
    med=df.loc[:,'time'].median()
    if med <600000 :
        interval_to_slice=df[(df['time'] < 1200000)]
    elif med >2400000 :
        interval_to_slice=df[(df['time'])>1800000]
    else :
        interval_to_slice=df[(df['time']>med-600000) & (df['time']<med+600000)]
    
    
    time_intervals = np.linspace(interval_to_slice['time'].to_numpy()[0], interval_to_slice['time'].to_numpy()[len(interval_to_slice)-1], num=num_frames + 1)
    for i in range(len(time_intervals) - 1):   
        start_time, end_time = time_intervals[i], time_intervals[i + 1]
        interval_data = interval_to_slice[(interval_to_slice['time'] >= start_time) & (interval_to_slice['time'] < end_time)]
        last_polarity_per_location =interval_data.groupby(['x', 'y'])['polarity'].sum().reset_index()
        last_pol_merged=pd.merge(DF2, last_polarity_per_location, on=['x', 'y'], how='left').fillna(0)
        scaler = MinMaxScaler()
        last_pol_merged['polarity']=scaler.fit_transform(last_pol_merged['polarity'].to_numpy().reshape(-1, 1))

        count_polarity_per_location =interval_data.groupby(['x', 'y'])['polarity'].count().reset_index()
        count_pol_merged=pd.merge(DF2, count_polarity_per_location, on=['x', 'y'], how='left').fillna(0)



        grouped_data = interval_data.groupby(['x', 'y']).agg({'polarity': nb_changes}).reset_index()
        DF_merged = pd.merge(DF2, grouped_data, on=['x', 'y'], how='left')
        DF_merged['polarity'].fillna(0, inplace=True)
        DF_merged['polarity'] = scaler.fit_transform(DF_merged[['polarity']])

        '''grouped_data2=pd.DataFrame(interval_data.groupby(['x', 'y']).agg({'polarity': list}).reset_index())
        DF_merged2 = pd.merge(DF2, grouped_data2, on=['x', 'y'], how='left')
        DF_merged2['polarity'].fillna(0, inplace=True)
        DF_merged2['polarity'] = scaler.fit_transform(DF_merged2.apply(lambda row: nb_positive_changes(row['polarity']), axis=1).to_numpy().reshape(-1, 1))
'''

        matrix1=last_pol_merged.pivot(index='y', columns='x', values='polarity').values 
        matrix2=count_pol_merged.pivot(index='y', columns='x', values='polarity').values 
        #matrix3=DF_merged2.pivot(index='y', columns='x', values='polarity').values ##nb of changes
        matrix4=DF_merged.pivot(index='y', columns='x', values='polarity').values ##nb of changes
            

        matrix3D = np.dstack((matrix1,matrix2,matrix4))
        results.append(matrix3D)
    all_results.append(results)        

In [ ]:
flattened_records =[matrix for lists in all_results for matrix in lists]
XX = np.array(flattened_records)
XX1 = XX.reshape((100,30, 90, 224,3))
XX3 = XX1[:,:, 15:65,50:170,:]
XX3[:,:,:,:,1]=XX3[:,:,:,:,1]/np.max(XX3[:,:,:,:,1])
XX3= downsize(0.6,XX3)



    


In [ ]:
correct_labels=['Addition',
 'Carnaval',
 'Decider',
 'Ecole',
 'Fillette',
 'Huitre',
 'Joyeux',
 'Musique',
 'Pyjama',
 'Ruisseau']

In [ ]:
np.save('testTensor',XX3)

In [ ]:
XX3=np.load('/kaggle/working/testTensor.npy')

In [ ]:
pred=[]
for i  in range (100) : 
    result=model.predict(XX3[i].reshape(1,30,30,72,3),verbose=None).argmax()
    pred.append(correct_labels[result])

In [ ]:
import os
import re

directory_path = '/kaggle/input/smemi309-final-evaluation-challenge-2023/test10/test10'

file_names = os.listdir(directory_path)

pattern = re.compile(r'\d+')

numbers = [int(pattern.search(file_name).group()) for file_name in file_names]




In [ ]:
id = numbers

data_list = list(zip(id, pred))
import csv
file_path = 'sampleSubmission.csv'
with open(file_path, 'w', newline='') as csvfile:
    csv_writer = csv.writer(csvfile)
    csv_writer.writerow(['id', 'label'])  # Header
    csv_writer.writerows(data_list)